In [30]:
# %pip install python-dotenv
# %pip install seaborn
# %pip install tensorflow_data_validation

In [31]:
import os
import pandas as pd
import geopandas as gpd
import pygeos as pg
import numpy as np
import sklearn as sk
import scipy as sp
import seaborn as sns
from IPython.display import clear_output
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

In [32]:
# The following lines adjust the granularity of reporting.
#pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir('F:\\Uni Files\\4710\\4710 Project\\MLweatherForestFire')
NULLFLAG = -9999
ONEDAY = 24 * 60 * 60
WEEK = 7 * ONEDAY

In [33]:
fireWeatherTable = "Data/FinalFeature.csv"
dfFireWeather = pd.read_csv(fireWeatherTable)
dfFireWeather.columns

Index(['ENTRYID', 'FIRE_ID', 'FIRENAME', 'YEAR', 'MONTH', 'DAY', 'REP_DATE',
       'SIZE_HA', 'SIZE_HA_BIN', 'GEOM', 'ELEVATIONM', 'DIST_TO_WATER',
       'CLIMATEID', 'PROVINCECODE', 'DAYW', 'MAXTEMP', 'MEANHUMIDITY',
       'MEANWINDSPEED', 'MAXWINDSPEED', 'TOTALPRECIP', 'RAIN', 'LONG', 'LAT',
       'LONGBIN', 'LATBIN'],
      dtype='object')

In [34]:
dailyWeather = "Data/WeatherDataHourlyAggDaily.csv"
dfWeather = pd.read_csv(dailyWeather)
dfWeather.columns

C:\Users\Red\AppData\Local\Temp\ipykernel_457804\2291072772.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  dfWeather = pd.read_csv(dailyWeather)


Index(['ClimateID', 'ProvinceCode', 'Year', 'Month', 'Day', 'MeanTemp',
       'MinTemp', 'MaxTemp', 'MeanDewPoint', 'MinDewPoint', 'MaxDewPoint',
       'MeanHumidity', 'MinHumidity', 'MaxHumidity', 'MeanPressure',
       'MinPressure', 'MaxPressure', 'MeanWindSpeed', 'MinWindSpeed',
       'MaxWindSpeed', 'MeanWindChill', 'MinWindChill', 'MaxWindChill',
       'TotalPrecip', 'MeanWindDirection'],
      dtype='object')

In [35]:
dfWeather.astype({'ClimateID': 'str', 'ProvinceCode': 'str', 
                'Year': 'int', 'Month': 'int', 'Day': 'int',
                'MeanTemp': 'float', 'MinTemp': 'float', 'MaxTemp': 'float',
                'MeanDewPoint': 'float', 'MinDewPoint': 'float', 'MaxDewPoint': 'float',
                'MeanHumidity': 'float', 'MinHumidity': 'float', 'MaxHumidity': 'float',
                'MeanPressure': 'float', 'MinPressure': 'float', 'MaxPressure': 'float',
                'MeanWindSpeed': 'float', 'MinWindSpeed': 'float', 'MaxWindSpeed': 'float',
                'MeanWindChill': 'float', 'MinWindChill': 'float', 'MaxWindChill': 'float',
                'TotalPrecip': 'float', 'MeanWindDirection': 'float'}, copy=False)
dfWeather.drop(columns=['MeanTemp', 'MinTemp', 'MeanDewPoint', 'MinDewPoint', 'MaxDewPoint',
                    'MinHumidity', 'MaxHumidity', 'MeanPressure', 'MinPressure',
                    'MaxPressure', 'MinWindSpeed', 'MeanWindChill', 'MinWindChill', 'MaxWindChill',
                    'MeanWindDirection' ], inplace=True)

Sums of max temp humidity days with precipitation in time period

In [36]:
dfWeatherDaily = dfWeather

In [37]:
# add utc from year month day
dfWeatherDaily['utc'] = pd.to_datetime(dfWeatherDaily[['Year', 'Month', 'Day']]).astype('int64')
dfFireWeather['utc'] = pd.to_datetime(dfFireWeather[['YEAR', 'MONTH', 'DAY']]).astype('int64')

In [38]:
dfWeatherDaily.count()

ClimateID        89909
ProvinceCode     89909
Year             89909
Month            89909
Day              89909
MaxTemp          89796
MeanHumidity     88632
MeanWindSpeed    88926
MaxWindSpeed     88926
TotalPrecip      89909
utc              89909
dtype: int64

In [45]:
# show weather 2015 4 28
dfWeatherDaily[(dfWeatherDaily['Year'] == 2015) & (dfWeatherDaily['Month'] == 4) & (dfWeatherDaily['Day'] == 28)]

,ClimateID,ProvinceCode,Year,Month,Day,MaxTemp,MeanHumidity,MeanWindSpeed,MaxWindSpeed,TotalPrecip,utc
5658,2202201,NT,2015,4,28,22.1,43.8,11.1,22.0,0.0,1430179200000000000
12647,2202202,NT,2015,4,28,22.7,42.0,7.6,14.0,0.0,1430179200000000000
27362,4060983,SK,2015,4,28,22.7,39.5,8.5,17.0,1.3,1430179200000000000
30468,4060984,SK,2015,4,28,22.8,31.8,12.6,22.0,0.0,1430179200000000000
32470,4060988,SK,2015,4,28,22.8,36.5,12.0,22.0,0.0,1430179200000000000
35073,4061620,SK,2015,4,28,11.4,68.0,12.0,19.0,0.0,1430179200000000000
46457,4063605,SK,2015,4,28,20.3,54.9,6.0,8.0,0.0,1430179200000000000
54199,4064149,SK,2015,4,28,21.5,52.2,7.2,14.0,0.0,1430179200000000000
62318,4064155,SK,2015,4,28,22.2,53.2,8.1,18.0,0.0,1430179200000000000
70174,4067655,SK,2015,4,28,13.6,66.7,4.6,9.0,0.0,1430179200000000000


In [39]:
# count rows with MaxTemp 0
dfWeatherDaily[dfWeatherDaily['MaxTemp'] == 0].count()

ClimateID        274
ProvinceCode     274
Year             274
Month            274
Day              274
MaxTemp          274
MeanHumidity     269
MeanWindSpeed    268
MaxWindSpeed     268
TotalPrecip      274
utc              274
dtype: int64

In [40]:
dfFWEW = dfFireWeather.copy(deep=True)

In [47]:
# add 7 day sum max temp column, 7 day sum humidity column, 7 day sum precip column
for index, row in dfFWEW.iterrows():
    fireDate = datetime(row['YEAR'], row['MONTH'], row['DAY'])
    maxTemps = []
    humidities = []
    windSpeeds = []
    rainBool = False

    for i in range(7):
        day = fireDate - timedelta(days=i)
        dayWeather = dfWeatherDaily.loc[(dfWeatherDaily['Year'] == day.year) & (dfWeatherDaily['Month'] == day.month) & (dfWeatherDaily['Day'] == day.day)]
        if dayWeather.empty:
            maxTemps.append(0)
            humidities.append(0)
            windSpeeds.append(0)
        else:
            maxTemps.append(dayWeather['MaxTemp'].max())
            humidities.append(dayWeather['MeanHumidity'].mean())
            windSpeeds.append(dayWeather['MeanWindSpeed'].mean())
            if dayWeather['TotalPrecip'].values[0] > 0:
                rainBool = True

    dfFWEW.at[index, '7daySumMaxTemp'] = sum(maxTemps)
    dfFWEW.at[index, '7daySumHumidity'] = sum(humidities)
    dfFWEW.at[index, '7daySumWindSpeed'] = sum(windSpeeds)
    # true if any day > 0
    dfFWEW.at[index, '7dayRain'] = rainBool
    

In [48]:
dfFWEW.count()

ENTRYID             836
FIRE_ID             836
FIRENAME            836
YEAR                836
MONTH               836
DAY                 836
REP_DATE            836
SIZE_HA             836
SIZE_HA_BIN         836
GEOM                836
ELEVATIONM          836
DIST_TO_WATER       836
CLIMATEID           836
PROVINCECODE        836
DAYW                836
MAXTEMP             836
MEANHUMIDITY        836
MEANWINDSPEED       836
MAXWINDSPEED        836
TOTALPRECIP         836
RAIN                836
LONG                836
LAT                 836
LONGBIN             836
LATBIN              836
utc                 836
7daySumMaxTemp      836
7daySumHumidity     836
7daySumWindSpeed    836
7dayRain            836
dtype: int64

In [49]:
dfFWEW[dfFWEW['7daySumMaxTemp'] == 0].count()

ENTRYID             0
FIRE_ID             0
FIRENAME            0
YEAR                0
MONTH               0
DAY                 0
REP_DATE            0
SIZE_HA             0
SIZE_HA_BIN         0
GEOM                0
ELEVATIONM          0
DIST_TO_WATER       0
CLIMATEID           0
PROVINCECODE        0
DAYW                0
MAXTEMP             0
MEANHUMIDITY        0
MEANWINDSPEED       0
MAXWINDSPEED        0
TOTALPRECIP         0
RAIN                0
LONG                0
LAT                 0
LONGBIN             0
LATBIN              0
utc                 0
7daySumMaxTemp      0
7daySumHumidity     0
7daySumWindSpeed    0
7dayRain            0
dtype: int64

In [ ]:
print(dfFWEW.count())
print(dfFWEW.describe())
print(dfFWEW.isnull().sum().sum())

In [50]:
# add 14 day sum max temp column, 14 day sum humidity column, 14 day sum precip column
for index, row in dfFWEW.iterrows():
    fireDate = datetime(row['YEAR'], row['MONTH'], row['DAY'])
    maxTemps = []
    humidities = []
    windSpeeds = []
    rainBool = False

    for i in range(14):
        day = fireDate - timedelta(days=i)
        dayWeather = dfWeatherDaily.loc[(dfWeatherDaily['Year'] == day.year) & (dfWeatherDaily['Month'] == day.month) & (dfWeatherDaily['Day'] == day.day)]
        if dayWeather.empty:
            maxTemps.append(0)
            humidities.append(0)
            windSpeeds.append(0)
        else:
            maxTemps.append(dayWeather['MaxTemp'].max())
            humidities.append(dayWeather['MeanHumidity'].mean())
            windSpeeds.append(dayWeather['MeanWindSpeed'].mean())
            if dayWeather['TotalPrecip'].values[0] > 0:
                rainBool = True

    dfFWEW.at[index, '14daySumMaxTemp'] = sum(maxTemps)
    dfFWEW.at[index, '14daySumHumidity'] = sum(humidities)
    dfFWEW.at[index, '14daySumWindSpeed'] = sum(windSpeeds)
    # true if any day > 0
    dfFWEW.at[index, '14dayRain'] = rainBool

In [ ]:
print(dfFWEW.count())
print(dfFWEW.describe())
print(dfFWEW.isnull().sum().sum())

In [ ]:
# save to csv
dfFWEW.to_csv('Data/FinalFeatureV3.csv', index=False)

In [ ]:
# group the fires with same utc and average the N day columns
dfFWEW = dfFWEW.groupby(['utc', 'YEAR', 'MONTH', 'DAY'], as_index=False).agg({'SIZE_HA':['sum'],
                                                     '7daySumMaxTemp':['mean'], '7daySumHumidity':['mean'], '7daySumPrecip':['mean'],
                                                     '14daySumMaxTemp':['mean'], '14daySumHumidity':['mean'], '14daySumPrecip':['mean'],
                                                     '21daySumMaxTemp':['mean'], '21daySumHumidity':['mean'], '21daySumPrecip':['mean'],
                                                     '28daySumMaxTemp':['mean'], '28daySumHumidity':['mean'], '28daySumPrecip':['mean'],
                                                     '35daySumMaxTemp':['mean'], '35daySumHumidity':['mean'], '35daySumPrecip':['mean'],
                                                     '42daySumMaxTemp':['mean'], '42daySumHumidity':['mean'], '42daySumPrecip':['mean'],
                                                     '49daySumMaxTemp':['mean'], '49daySumHumidity':['mean'], '49daySumPrecip':['mean'],
                                                     '56daySumMaxTemp':['mean'], '56daySumHumidity':['mean'], '56daySumPrecip':['mean']})

In [ ]:
print(dfFWEW.count())
print(dfFWEW.describe())
print(dfFWEW.isnull().sum().sum())

In [ ]:
dfEval = dfFWEW

In [ ]:
# randomly select 6 years from 2010-2019 for training
dfTrain = dfEval[dfEval['YEAR'].isin([2010, 2011, 2012, 2013, 2014, 2015, 2016])]
dfValidate = dfEval[dfEval['YEAR'].isin([2017, 2018])]
dfTest = dfEval[dfEval['YEAR'].isin([2019, 2020])]

In [ ]:
# Store our random selection, run once
randomTrain = "RandomTrain"
dfTrain.to_sql(randomTrain, db_push_con, if_exists='replace', index=False)

randomTest = "RandomTest"
dfTest.to_sql(randomTest, db_push_con, if_exists='replace', index=False)

randomValidate = "RandomValidate"
dfValidate.to_sql(randomValidate, db_push_con, if_exists='replace', index=False)

In [ ]:
trainStats = tfdv.generate_statistics_from_dataframe(dfTrain)

In [ ]:
tfdv.visualize_statistics(trainStats)


In [ ]:
schema = tfdv.infer_schema(statistics=trainStats)
tfdv.display_schema(schema=schema)


In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn import svm
from sklearn.inspection import DecisionBoundaryDisplay


Y_train = dfTrain['SIZE_HA'].copy(deep=True)
Y_train.fillna(Y_train.mean(), inplace=True)

Y_train_discrete = dfTrain['size_ha_bin'].copy(deep=True)
Y_train_discrete.fillna(Y_train_discrete.min(), inplace=True)



X_train = dfTrain.drop(columns=['MONTH', 'SIZE_HA', 'OneMonth', 'OneYear', 'TwoMonth', 'TwoYear', 'EntryID', 'size_ha_bin', 'YEAR', 'DAY', 'FIRE_ID', 'FIRENAME', 'ClimateID', 'REP_DATE'])
X_train.fillna(X_train.mean(), inplace=True)
X_train_one = X_train.drop(columns=['TwoMeanTemp', 'TwoMinTemp', 'TwoMaxTemp', 'TwoMeanDewPoint', 'TwoMinDewPoint', 'TwoMaxDewPoint',
                                     'TwoMeanHumidity', 'TwoMinHumidity', 'TwoMaxHumidity', 'TwoMeanPressure', 'TwoMinPressure',
                                     'TwoMaxPressure', 'TwoMeanWindSpeed', 'TwoMinWindSpeed', 'TwoMaxWindSpeed', 'TwoMeanWindChill',
                                     'TwoMinWindChill', 'TwoMaxWindChill', 'TwoTotalPrecip', 'TwoMeanWindDirection'])
X_train_two = X_train.drop(columns=['OneMeanTemp', 'OneMinTemp', 'OneMaxTemp', 'OneMeanDewPoint', 'OneMinDewPoint', 'OneMaxDewPoint',
                                     'OneMeanHumidity', 'OneMinHumidity', 'OneMaxHumidity', 'OneMeanPressure', 'OneMinPressure',
                                     'OneMaxPressure', 'OneMeanWindSpeed', 'OneMinWindSpeed', 'OneMaxWindSpeed', 'OneMeanWindChill',
                                      'OneMinWindChill', 'OneMaxWindChill', 'OneTotalPrecip', 'OneMeanWindDirection'])

X_train_means_one = X_train_one.drop(columns=['OneMinTemp', 'OneMaxTemp', 'OneMinDewPoint', 'OneMaxDewPoint', 'OneMinHumidity', 'OneMaxHumidity',
                                              'OneMinPressure', 'OneMaxPressure', 'OneMinWindSpeed', 'OneMaxWindSpeed', 'OneMinWindChill',
                                              'OneMaxWindChill'])

X_train_means_two = X_train_two.drop(columns=['TwoMinTemp', 'TwoMaxTemp', 'TwoMinDewPoint', 'TwoMaxDewPoint', 'TwoMinHumidity', 'TwoMaxHumidity',
                                                'TwoMinPressure', 'TwoMaxPressure', 'TwoMinWindSpeed', 'TwoMaxWindSpeed', 'TwoMinWindChill',
                                                'TwoMaxWindChill'])

dfTrainScaled = dfTrain.copy(deep=True)
dfTrainScaled.fillna(dfTrainScaled.mean(), inplace=True)
dfTrainScaled = dfTrainScaled.drop(columns=['MONTH', 'OneMonth', 'OneYear', 'TwoMonth', 'TwoYear', 'EntryID', 'size_ha_bin', 'YEAR', 'DAY', 'FIRE_ID', 'FIRENAME', 'ClimateID', 'REP_DATE'])

# regularize y using log scale
Y_train = np.log(Y_train)
# regularize y values using z score
Y_train = (Y_train - Y_train.mean()) / Y_train.std()
# set max value to 3 zscore
Y_train[Y_train > 3] = 3

In [ ]:
dfTrainScaled = dfTrain.copy(deep=True)

In [ ]:
dfTrainScaled['SIZE_HA'] = np.log(dfTrainScaled['SIZE_HA'])
# regularize y values using z score
dfTrainScaled = (dfTrainScaled - dfTrainScaled.mean()) / dfTrainScaled.std()
# set max value to 3 zscore
dfTrainScaled[dfTrainScaled > 3] = 3
# set min value to -3 zscore
dfTrainScaled[dfTrainScaled < -3] = -3

# shift the wole train set to be positive
dfTrainScaled = dfTrainScaled + 3

In [ ]:
trainStats2 = tfdv.generate_statistics_from_dataframe(dfTrainScaled)
tfdv.visualize_statistics(trainStats2)

In [ ]:
# one week data
weekData = pd.DataFrame()
train_dataset = dfTrainScaled.copy(deep=True)

# create a dataframe with the 7 day data
weekData['7daySumMaxTemp'] = train_dataset['7daySumMaxTemp']
weekData['7daySumHumidity'] = train_dataset['7daySumHumidity']
weekData['7daySumPrecip'] = train_dataset['7daySumPrecip']
weekData['SIZE_HA'] = train_dataset['SIZE_HA']


pairplotOne = sns.pairplot(weekData, kind="reg", diag_kind="kde")

In [ ]:
# one month data
monthData = pd.DataFrame()
train_dataset = dfTrainScaled.copy(deep=True)

# create a dataframe with the 28 day data
monthData['28daySumMaxTemp'] = train_dataset['28daySumMaxTemp']
monthData['28daySumHumidity'] = train_dataset['28daySumHumidity']
monthData['28daySumPrecip'] = train_dataset['28daySumPrecip']
monthData['SIZE_HA'] = train_dataset['SIZE_HA']

pairplotTwo = sns.pairplot(monthData, kind="reg", diag_kind="kde")

In [ ]:
# two month data
twoMonthData = pd.DataFrame()
train_dataset = dfTrainScaled.copy(deep=True)

# create a dataframe with the 56 day data
twoMonthData['56daySumMaxTemp'] = train_dataset['56daySumMaxTemp']
twoMonthData['56daySumHumidity'] = train_dataset['56daySumHumidity']
twoMonthData['56daySumPrecip'] = train_dataset['56daySumPrecip']
twoMonthData['SIZE_HA'] = train_dataset['SIZE_HA']

pairplotThree = sns.pairplot(twoMonthData, kind="reg", diag_kind="kde")

In [ ]:
print(dfTrainScaled['SIZE_HA'])


In [ ]:
dfTemp = pd.DataFrame()
dfTemp['SIZE_HA'] = dfTrainScaled['SIZE_HA'].copy(deep=True)
# categorize size_ha into 4 classes by quantile
dfTrainScaled['SIZE_BIN'] = pd.qcut(dfTemp['SIZE_HA'], 4, labels=False)

In [ ]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(dfTrainScaled.drop(columns=['SIZE_HA']), dfTrainScaled['SIZE_BIN'])

In [ ]:

print(dfTrainScaled.count())
print(dfTrainScaled.isna().sum().sum())
print(dfTrainScaled.dtypes)

In [ ]:
C = 0.9  # SVM regularization parameter
models = (
    svm.SVC(kernel="linear", C=C, decision_function_shape='ovo'),
    svm.LinearSVC(C=C, max_iter=10000),
    svm.SVC(kernel="rbf", gamma=0.7, C=C, decision_function_shape='ovo'),
    svm.SVC(kernel="poly", degree=3, gamma="auto", C=C, decision_function_shape='ovo'),
)
dfTemp = pd.DataFrame()
dfTemp['28daySumMaxTemp'] = dfTrainScaled['28daySumMaxTemp'].copy(deep=True)
dfTemp['28daySumHumidity'] = dfTrainScaled['28daySumHumidity'].copy(deep=True)
models = (clf.fit(dfTemp, dfTrainScaled['SIZE_BIN']) for clf in models)

# title for the plots
titles = (
    "SVC with linear kernel",
    "LinearSVC (linear kernel)",
    "SVC with RBF kernel",
    "SVC with polynomial (degree 3) kernel",
)

# Set-up 2x2 grid for plotting.
fig, sub = plt.subplots(2, 2, figsize=(19.20, 10.80))
plt.subplots_adjust(wspace=0.1, hspace=0.2)

X0, X1 = dfTemp['28daySumMaxTemp'], dfTemp['28daySumHumidity']

for clf, title, ax in zip(models, titles, sub.flatten()):
    disp = DecisionBoundaryDisplay.from_estimator(
        clf,
        dfTemp,
        response_method="predict",
        cmap=plt.cm.coolwarm,
        alpha=0.8,
        ax=ax,
        xlabel=['28daySumMaxTemp', '28daySumHumidity'],
        ylabel="SIZE_BIN",
    )
    ax.scatter(X0, X1, c=dfTrainScaled['SIZE_BIN'], cmap=plt.cm.coolwarm, s=20, edgecolors="k")
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)

plt.show()

## Define functions that build and train a model

The following code defines two functions:

  * `build_model(my_learning_rate)`, which builds a randomly-initialized model.
  * `train_model(model, feature, label, epochs)`, which trains the model from the examples (feature and label) you pass. 

Since you don't need to understand model building code right now, we've hidden this code cell.  You may optionally double-click the following headline to see the code that builds and trains a model.

In [ ]:
#@title Define the functions that build and train a model
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Describe the topography of the model.
  # The topography of a simple linear regression model
  # is a single node in a single layer.
  model.add(tf.keras.layers.Dense(units=1, 
                                  input_shape=(1,)))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model        


def train_model(model, df, feature, label, epochs, batch_size):
  """Train the model by feeding it data."""

  # Feed the model the feature and the label.
  # The model will train for the specified number of epochs. 
  history = model.fit(x=df[feature],
                      y=df[label],
                      batch_size=batch_size,
                      epochs=epochs)

  # Gather the trained model's weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the rest of history.
  epochs = history.epoch
  
  # Isolate the error for each epoch.
  hist = pd.DataFrame(history.history)

  # To track the progression of training, we're going to take a snapshot
  # of the model's root mean squared error at each epoch. 
  rmse = hist["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse

print("Defined the build_model and train_model functions.")

## Define plotting functions

The following [matplotlib](https://developers.google.com/machine-learning/glossary/#matplotlib) functions create the following plots:

*  a scatter plot of the feature vs. the label, and a line showing the output of the trained model
*  a loss curve

You may optionally double-click the headline to see the matplotlib code, but note that writing matplotlib code is not an important part of learning ML programming.

In [ ]:
#@title Define the plotting functions
def plot_the_model(trained_weight, trained_bias, feature, label):
  """Plot the trained model against 200 random training examples."""

  # Label the axes.
  plt.xlabel(feature)
  plt.ylabel(label)

  # Create a scatter plot from 200 random points of the dataset.
  random_examples = dfTrainScaled.sample(n=200)
  plt.scatter(random_examples[feature], random_examples[label])

  # Create a red line representing the model. The red line starts
  # at coordinates (x0, y0) and ends at coordinates (x1, y1).
  x0 = 0
  y0 = trained_bias
  x1 = 6
  y1 = trained_bias + (trained_weight * x1)
  plt.plot([x0, x1], [y0, y1], c='r')

  # Render the scatter plot and the red line.
  plt.show()


def plot_the_loss_curve(epochs, rmse):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()  

print("Defined the plot_the_model and plot_the_loss_curve functions.")

## Call the model functions

An important part of machine learning is determining which [features](https://developers.google.com/machine-learning/glossary/#feature) correlate with the [label](https://developers.google.com/machine-learning/glossary/#label). For example, real-life home-value prediction models typically rely on hundreds of features and synthetic features. However, this model relies on only one feature. For now, you'll arbitrarily use `total_rooms` as that feature. 


In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.01
epochs = 30
batch_size = 5

# Specify the feature and the label.
my_feature = "OneMeanHumidity"  # the total number of rooms on a specific city block.
my_label="SIZE_HA" # the median value of a house on a specific city block.
#my_label="size_ha_bin"
# That is, you're going to create a model that predicts house value based 
# solely on total_rooms.  

# Discard any pre-existing version of the model.
my_model = None

# Invoke the functions.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, dfTrainScaled, 
                                         my_feature, my_label,
                                         epochs, batch_size)

print("\nThe learned weight for your model is %.4f" % weight)
print("The learned bias for your model is %.4f\n" % bias )

plot_the_model(weight, bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

## Use the model to make predictions

You can use the trained model to make predictions. In practice, [you should make predictions on examples that are not used in training](https://developers.google.com/machine-learning/crash-course/training-and-test-sets/splitting-data). However, for this exercise, you'll just work with a subset of the same training dataset. A later Colab exercise will explore ways to make predictions on examples not used in training.

First, run the following code to define the house prediction function:

In [ ]:
def predict_house_values(n, feature, label):
  """Predict house values based on a feature."""

  batch = dfTrainScaled[feature][200:200 + n]
  predicted_values = my_model.predict_on_batch(x=batch)

  print("feature   label          predicted")
  print("  value   value          value")
  print("          in thousand$   in thousand$")
  print("--------------------------------------")
  for i in range(n):
    print ("%5.0f %6.0f %15.0f" % (dfTrain[feature][400+i], dfTrain[label][400+i], predicted_values[i][0] ))

In [ ]:
predict_house_values(10, my_feature, my_label)